# Libraries

In [1]:
import pyodbc
import pandas as pd
from openpyxl import load_workbook

# Database settings

In [2]:
# Connect to the database
server = 'DESKTOP-HTHQ40O\SQLEXPRESS'
database = 'SouthAfrica_CrimeStats'
username = 'cnoltehj'
password = 'HeidiJoy@1_2024'


# Validate Policestation and Exclude rows

In [ ]:
# Load the Excel file
#file_path = "./Dataset_CSV_Files/2023_2024/2023-2024-1st_Quarter_WEB.xlsx"

#file_path = "./Dataset_CSV_Files/2023_2024/2023-2024_-_2nd_Quarter_WEB.xlsx"

#file_path = "./Dataset_CSV_Files/2023_2024/2023-2024_-_3rd_Quarter_WEB.xlsx"

file_path = "./Dataset_CSV_Files/2023_2024/2023-2024_-_4th_Quarter_WEB.xlsx" 

wb = load_workbook(file_path, data_only=True)

# Access the desired sheet
# Q-1, 2,3,4
sheet = wb["Crime stats per component"]

# Quarter
#sheet = wb["Crime stats per component"]

# Check the value of cell Q8
cell_l8 = sheet["L8"].value

# Check the value of cell Q9
cell_l9 = sheet["L9"].value

# Q 2,3,4
excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 23, 28, 36, 40, 47, 52, 54, 55, 61, 62]
excluded_columns = [1, 2, sheet.max_column - 1, sheet.max_column]  # First 2 columns and last 2 columns

print("Province : ",cell_l8)
print("Police Station : ",cell_l9)

#     # Iterate over all rows except for the excluded ones
# for row_number in range(2, sheet.max_row + 1):
#     if row_number not in excluded_rows:
#             # Retrieve row values and exclude specified columns
#         row_values = [cell.value for col_num, cell in enumerate(sheet[row_number]) if col_num + 1 not in excluded_columns]

#             # Print the row values as a single dataset
#         print(row_values)



Province :  Western Cape
Police Station :  Knysna


# Insert Stats 2019 -2023 into Database

In [192]:
# Define the province parameter
selected_province = "ZA.WC"
selected_policestation = "KA63WC"
selected_quarter = 4

cnxn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = cnxn.cursor()

# Iterate over all rows except excluded ones
for row_number in range(3, sheet.max_row + 1):  # Start from the third row
    
    if row_number not in excluded_rows:
        # Retrieve row values and exclude specified columns
        row_values = [cell.value for col_num, cell in enumerate(sheet[row_number]) if col_num + 1 not in excluded_columns]
        
        # Ensure row_values has enough elements before accessing
        #if len(row_values) > 1:
            # Convert the elements to int if they represent numeric values
        for i in range(1, 1):
            try:
                    row_values[i] = int(row_values[i])
            except (ValueError, IndexError):
                    # Handle the case where the value cannot be converted to int or index error occurs
                pass  # Handle this case as per your requirement
                
            # Inside the loop DB
        try:
                cursor.execute("EXEC sp_Add_QuarterlyCrimeStats ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?",
                   (str(row_values[0]).strip(), row_values[1], row_values[2], row_values[3], row_values[4], row_values[5], row_values[6], str(row_values[7]).strip(), str(selected_province).strip(), str(selected_policestation).strip(), int(selected_quarter)))
                print('Input 1')
                
                # Commit the changes to the database
                cnxn.commit()

                # Print row_values for debugging
                print("Row values:", row_values)

        except pyodbc.Error as e:
                print("Error executing SQL query:", e)


# Close the cursor and connection
cursor.close()
cnxn.close()

Input 1
Row values: ['Murder', 6, 4, 7, 7, 3, -4, '4 counts lower']
Input 1
Row values: ['Sexual offences', 42, 32, 39, 27, 24, -3, '3 counts lower']
Input 1
Row values: ['Attempted murder', 3, 3, 9, 9, 7, -2, '2 counts lower']
Input 1
Row values: ['Assault with the intent to inflict grievous bodily harm', 116, 109, 127, 142, 182, 40, 0.2816901408450705]
Input 1
Row values: ['Common assault', 176, 161, 149, 141, 194, 53, 0.375886524822695]
Input 1
Row values: ['Common robbery', 34, 23, 21, 27, 45, 18, '18 counts higher']
Input 1
Row values: ['Robbery with aggravating circumstances', 42, 44, 19, 41, 47, 6, '6 counts higher']
Input 1
Row values: ['Contact crime (Crimes against the person)', 419, 376, 371, 394, 502, 108, 0.2741116751269035]
Input 1
Row values: ['Rape', 30, 21, 28, 20, 16, -4, '4 counts lower']
Input 1
Row values: ['Sexual assault', 10, 4, 5, 3, 5, 2, '2 counts higher']
Input 1
Row values: ['Attempted sexual offences', 2, 4, 3, 3, 2, -1, '1 count lower']
Input 1
Row values

# Get Extra years for 2016 sheets and exclude rows

In [13]:
# Load the Excel file

#file_path = "./Dataset_CSV_Files/2020_2021/first_quarter 2020_2021_crime_statistics.xlsx"

#file_path = "./Dataset_CSV_Files/2020_2021/crime_statistics_july_september_2020_21.xlsx"

#file_path = "./Dataset_CSV_Files/2020_2021/crime_statistics_third_qaurter 2020_2021.xlsx"

file_path = "./Dataset_CSV_Files/2020_2021/crime_statistics_fourth_qaurter 2020_2021_current.xlsx"

#wb = load_workbook(file_path_q1_2016_2018, data_only=True)

wb = load_workbook(file_path, data_only=True)

#wb = load_workbook(file_path_q3_2016_2018, data_only=True)

#wb = load_workbook(file_path_q4_2016_2018, data_only=True)

# Access the desired sheet
# 1 Quarter
#sheet = wb["Dropdown tables stations"]

# 2 Quarter
#sheet = wb["Stations per province"]

# 3 Quarter
#sheet = wb["Quarterly stations p province"]

# 4 Quarter
sheet = wb["Quarterly stations per province"]

# Check the value of cell Q8
cell_j9 = sheet["J9"].value

# Check the value of cell Q9
cell_j10 = sheet["J10"].value

# 1 Quarter
#excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 22, 28, 29, 36, 40, 46, 53]

# 2 Quarter
#excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 22, 28, 35, 39, 46, 52]

#3 Quarter and 4 Quarter
excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 22, 29, 36, 40, 47, 53]

excluded_columns = [1, 2, sheet.max_column - 1, sheet.max_column]  # First 2 columns and last 2 columns

print("Province : ",cell_j9)
print("Police Station : ",cell_j10)

#     # Iterate over all rows except for the excluded ones
# for row_number in range(2, sheet.max_row + 1):
#     if row_number not in excluded_rows:
#             # Retrieve row values and exclude specified columns
#         row_values = [cell.value for col_num, cell in enumerate(sheet[row_number]) if col_num + 1 not in excluded_columns]

            # Print the row values as a single dataset
        #print(row_values)
     

Province :  WESTERN_CAPE
Police Station :  KNYSNA


# Insert Extra years into database

In [14]:
import pyodbc
from openpyxl import load_workbook

# Connect to the database
server = 'DESKTOP-HTHQ40O\SQLEXPRESS'
database = 'SouthAfrica_CrimeStats'
username = 'cnoltehj'
password = 'HeidiJoy@1_2024'
cnxn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = cnxn.cursor()

# Define the province parameter
selected_province = "ZA.WC"
selected_policestation = "KA63WC"
selected_quarter = 4

# Iterate over all rows except excluded ones
for row_number in range(3, sheet.max_row + 1):  # Start from the third row
    
    if row_number not in excluded_rows:
        # Retrieve row values and exclude specified columns
        row_values = [cell.value for col_num, cell in enumerate(sheet[row_number]) if col_num + 1 not in excluded_columns]
        
        # Ensure row_values has enough elements before accessing
        #if len(row_values) > 1:
            # Convert the elements to int if they represent numeric values
        for i in range(1, 1):
            try:
                    row_values[i] = int(row_values[i])
            except (ValueError, IndexError):
                    # Handle the case where the value cannot be converted to int or index error occurs
                pass  # Handle this case as per your requirement
                
            # Inside the loop
        try:
                cursor.execute("EXEC sp_Add_QuarterlyCrimeStats_PreviousYears ?, ?, ?, ?, ?, ?, ?",
                   (str(row_values[0]).strip(), row_values[1], row_values[2], row_values[3], str(selected_province).strip(), str(selected_policestation).strip(), int(selected_quarter)))
                

                # Commit the changes to the database
                cnxn.commit()

                # Print row_values for debugging
                #print("Row values:", row_values)

        except pyodbc.Error as e:
                print("Error executing SQL query:", e)


# Close the cursor and connection
cursor.close()
cnxn.close()